CSCE 4143: Data Mining

Group 7 Final Project

UFC Stuff

https://www.kaggle.com/datasets/akshaysinghim/ufc-fight-data-1993-to-2023

Levi Crider, Spencer Smith, Caleb Holmes, and Ethan Weems

## Installing Dependencies

In [ ]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

In [ ]:
import pandas as pd
# Load data
df = pd.read_csv("data/ufc_data.csv")

In [ ]:
df = df.rename(columns={'R_fighter': 'RED'})
df = df.rename(columns={'B_fighter': 'BLUE'})
df = df.rename(columns={'R_SIG_STR_pct': 'R_SIG_PCT'})
df = df.rename(columns={'B_SIG_STR_pct': 'B_SIG_PCT'})
df = df.rename(columns={'R_TD_pct': 'R_TD_PCT'})
df = df.rename(columns={'B_TD_pct': 'B_TD_PCT'})
df = df.rename(columns={'win_by': 'WIN_TYPE'})
df = df.rename(columns={'last_round': 'LAST_RD'})
df = df.rename(columns={'last_round_time': 'LAST_RD_TIME'})
df = df.rename(columns={'Format': 'NUM_RD'})
df = df.rename(columns={'Referee': 'REF'})
df = df.rename(columns={'date': 'DATE'})
df = df.rename(columns={'location': 'LOC'})
df = df.rename(columns={'Fight_type': 'TYPE'})
df = df.rename(columns={'Winner': 'WINNER'})


# Cleaning Data

In [ ]:
# Dropping all rows with missing values
df.replace("---", pd.NaT, inplace=True)
df.dropna()



# Dropping all rows before may of 2001
df['DATE'] = pd.to_datetime(df['DATE'], format="%B %d, %Y").dt.strftime("%Y-%m-%d")
limit_date = '2001-04-01'
df = df[(df['DATE'] > limit_date)]



In [ ]:
# Remove percentage signs from columns
df = df.replace('%', '', regex=True)

In [ ]:
# Combine last round and last round time into one column called total_time


# Decision Tree Classifier